In [1]:
import os
import tempfile

import anndata as ad
import muon
import numpy as np
import scanpy as sc
import seaborn as sns
import torch
import muon as mu
import pertpy as pt
import pandas as pd
import re


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
import scvi
print(scvi.__version__)


1.1.2


In [3]:
# Load the .h5ad file
adata = ad.read_h5ad('/Users/ismail/thesis/Baf.single_gene.guide_qc.lm.h5ad')

# Display the object
print(adata)


AnnData object with n_obs × n_vars = 38328 × 1289
    obs: 'n_genes', 'batch', 'cell', 'control', 'guide.compact', 'gene.compact', 'unassigned', 'n_genes_by_counts', 'total_counts_mt', 'leiden', 'cell_state', 'SS18_4', 'BCL7A_4', 'SMARCE1_4', 'SS18L_1', 'BCL7B_2', 'SMARCA4_5', 'ARID2_3', 'SMARCD3_3', 'SMARCA2_1', 'BRD9_4', 'GLTSCR1_4', 'SMARCC2_3', 'SS18L_3', 'PHF10_3', 'NTC2_IG216151', 'DPF2_3', 'SS18L_2', 'ARID2_2', 'BCL7C_2', 'PBRM1_3', 'BCL7A_2', 'ARID2_1', 'BCL7B_3', 'GLTSCR1L_4', 'SMARCC2_2', 'SMARCA2_2', 'BRD7_4', 'DPF3_3', 'SMARCB1_5', 'BRD9_2', 'ARID2_4', 'GLTSCR1L_2', 'PHF10_4', 'SMARCA4_1', 'SMARCA4_4', 'ACTL6A_2', 'SS18L_4', 'BCL7B_4', 'SMARCD2_4', 'SMARCD1_2', 'NTC7_IG1144056', 'DPF3_2', 'SMARCA4_7', 'BRD7_3', 'BCL7B_1', 'DPF1_2', 'SMARCD2_2', 'ARID1A_2', 'PHF10_2', 'SMARCC1_7', 'NTC5_mNTC100A67005', 'BCL7A_3', 'SMARCD2_1', 'SMARCD2_3', 'SMARCE1_2', 'BCL7C_3', 'SMARCB1_7', 'BRD7_2', 'SS18_3', 'ACTL6B_1', 'BRD9_1', 'ACTL6A_1', 'SMARCC1_1', 'PBRM1_2', 'ARID1B_4', 'GLTSCR1_1'

In [4]:
adata.obs


,n_genes,batch,cell,control,guide.compact,gene.compact,unassigned,n_genes_by_counts,total_counts_mt,leiden,...,.P2,.P3,.P4,.P5,.P6,.P7,.P8,.P9,.P10,.P11
TTGCATTTCGGTAGAG-1-8,4364,8,0.0,0.0,SMARCA2_3,SMARCA2,0,4364,1777.0,2,...,0.209884,0.091023,-0.079784,-0.052859,-0.023871,-0.106268,-0.065490,-0.093925,-0.138877,-0.126892
TTTGGTTGTAGGTTTC-1-6,4523,6,0.0,0.0,unassigned,unassigned,1,4523,1697.0,4,...,0.303646,0.034636,-0.066206,-0.037091,-0.135707,-0.066927,-0.108650,-0.121789,-0.145670,-0.134484
CACGTGGCAGGCATGA-1-8,3016,8,0.0,0.0,BCL7B_1,BCL7B,0,3016,896.0,5,...,-0.274533,0.024449,0.149817,-0.087757,-0.088601,-0.031453,-0.077011,-0.125244,-0.046160,-0.135437
ACCAAACGTTGAAGTA-1-12,5928,12,0.0,0.0,unassigned,unassigned,1,5928,2412.0,3,...,-0.158908,0.048270,-0.066180,-0.016550,-0.008066,-0.044271,-0.119790,-0.104319,-0.118289,-0.167694
CTCAGAAAGACTCTAC-1-11,4594,11,0.0,0.0,ACTL6B_4,ACTL6B,0,4594,1859.0,1,...,0.190171,-0.011801,-0.069991,-0.067258,-0.045424,-0.100850,-0.142342,-0.088255,-0.152857,-0.147155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AGAAGCGTCAGGACGA-1-9,4964,9,0.0,0.0,PBRM1_4,PBRM1,0,4963,2287.0,3,...,-0.124360,0.092573,-0.120545,-0.052604,-0.071641,-0.061278,-0.111678,-0.107756,-0.133673,-0.133523
GTTGCGGCAACCCGCA-1-6,6043,6,0.0,0.0,BCL7A_1,BCL7A,0,6043,3493.0,3,...,0.045556,0.042191,-0.087506,-0.045139,-0.055465,0.004878,-0.104217,-0.095756,-0.132171,-0.180121
GGAGATGAGTAGAATC-1-0,3582,0,0.0,0.0,unassigned,unassigned,1,3582,1196.0,0,...,0.312022,0.013156,0.010845,-0.054402,-0.055282,-0.136859,-0.161164,-0.071691,-0.107270,-0.123363
TCTTTGAGTCAGTTTG-1-6,4472,6,0.0,0.0,SS18_3,SS18,0,4472,1646.0,4,...,0.304349,0.027444,-0.038663,-0.064119,-0.075211,-0.041830,-0.099594,-0.116923,-0.120978,-0.112966


In [5]:
adata.var

,n_cells,mt,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts,highly_variable,means,dispersions,dispersions_norm,mean,std,coef.gene2.program,coef.gene2.program_color,new_program,new_program.color
MUC1,15025,False,15025,0.347607,73.686054,19848.0,True,0.165828,-0.308917,0.504095,1.732787e-11,0.237323,9,#fc8d62,P7,#66c2a5
CCR1,7732,False,7732,0.177831,86.458607,10154.0,True,0.093780,0.003922,1.731725,1.279599e-11,0.193050,7,#b3b3b3,P11,#fc8d62
RBKS,4499,False,4499,0.088460,92.120703,5051.0,True,0.047697,-0.285813,0.594757,-2.428771e-11,0.132941,7,#b3b3b3,P11,#fc8d62
F3,25422,False,25422,0.945831,55.477329,54006.0,True,0.396673,0.229081,2.783264,-2.281654e-13,0.389663,1,#fc8d62,P3,#8da0cb
TXNIP,33982,False,33982,1.456751,40.485823,83179.0,True,0.583369,0.616224,3.883252,3.586021e-11,0.458277,9,#fc8d62,P7,#66c2a5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MCUB,9540,False,9540,0.204767,83.292177,11692.0,True,0.105099,-0.290108,0.577905,-3.278312e-12,0.194509,9,#fc8d62,P7,#66c2a5
ABCA1,1220,False,1220,0.023976,97.863360,1369.0,True,0.013932,0.017293,1.784193,6.355938e-12,0.075058,6,#e5c494,P9,#e78ac3
FEZ1,3985,False,3985,0.121911,93.020894,6961.0,True,0.074822,0.443046,3.454918,-1.630696e-11,0.191987,6,#e5c494,P9,#e78ac3
NBR1,11367,False,11367,0.245153,80.092471,13998.0,True,0.120634,-0.271094,0.652518,-1.103511e-12,0.204499,9,#fc8d62,P7,#66c2a5


In [6]:
def extract_guide_number(guide):
    if guide == 'unassigned':
        return guide
    else:
        # Extract the numeric part after the underscore
        match = re.search(r'_(\d+)', guide)
        if match:
            return match.group(1)
        else:
            return np.nan

adata.obs['guide.compact'] = adata.obs['guide.compact'].apply(extract_guide_number)
adata.obs["guide.compact"]

TTGCATTTCGGTAGAG-1-8              3
TTTGGTTGTAGGTTTC-1-6     unassigned
CACGTGGCAGGCATGA-1-8              1
ACCAAACGTTGAAGTA-1-12    unassigned
CTCAGAAAGACTCTAC-1-11             4
                            ...    
AGAAGCGTCAGGACGA-1-9              4
GTTGCGGCAACCCGCA-1-6              1
GGAGATGAGTAGAATC-1-0     unassigned
TCTTTGAGTCAGTTTG-1-6              3
GCTACAAGTATACCCA-1-2              1
Name: guide.compact, Length: 38328, dtype: object

In [7]:
adata.obs["gene.compact"]

TTGCATTTCGGTAGAG-1-8        SMARCA2
TTTGGTTGTAGGTTTC-1-6     unassigned
CACGTGGCAGGCATGA-1-8          BCL7B
ACCAAACGTTGAAGTA-1-12    unassigned
CTCAGAAAGACTCTAC-1-11        ACTL6B
                            ...    
AGAAGCGTCAGGACGA-1-9          PBRM1
GTTGCGGCAACCCGCA-1-6          BCL7A
GGAGATGAGTAGAATC-1-0     unassigned
TCTTTGAGTCAGTTTG-1-6           SS18
GCTACAAGTATACCCA-1-2        GLTSCR1
Name: gene.compact, Length: 38328, dtype: category
Categories (30, object): ['ACTL6A', 'ACTL6B', 'ARID1A', 'ARID1B', ..., 'SS18', 'SS18L', 'control', 'unassigned']

In [8]:
perturbed_adata = adata[
    adata.obs["gene.compact"] != "unassigned"
]  # Only consider perturbed cells
perturbed_adata.obs


,n_genes,batch,cell,control,guide.compact,gene.compact,unassigned,n_genes_by_counts,total_counts_mt,leiden,...,.P2,.P3,.P4,.P5,.P6,.P7,.P8,.P9,.P10,.P11
TTGCATTTCGGTAGAG-1-8,4364,8,0.0,0.0,3,SMARCA2,0,4364,1777.0,2,...,0.209884,0.091023,-0.079784,-0.052859,-0.023871,-0.106268,-0.065490,-0.093925,-0.138877,-0.126892
CACGTGGCAGGCATGA-1-8,3016,8,0.0,0.0,1,BCL7B,0,3016,896.0,5,...,-0.274533,0.024449,0.149817,-0.087757,-0.088601,-0.031453,-0.077011,-0.125244,-0.046160,-0.135437
CTCAGAAAGACTCTAC-1-11,4594,11,0.0,0.0,4,ACTL6B,0,4594,1859.0,1,...,0.190171,-0.011801,-0.069991,-0.067258,-0.045424,-0.100850,-0.142342,-0.088255,-0.152857,-0.147155
CATCAAGCAAGCGGAT-1-4,4865,4,0.0,0.0,2,DPF3,0,4865,2032.0,6,...,-0.044306,0.083702,-0.101828,0.053535,-0.028463,-0.046678,-0.105566,-0.110459,-0.131653,-0.159632
ATTCTTGAGTTTCAGC-1-2,4097,2,0.0,0.0,3,BCL7B,0,4097,1335.0,3,...,0.111221,0.047590,-0.041850,0.062592,-0.139507,0.079137,-0.095947,-0.136911,-0.075055,-0.138381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGGATACAATAAGGT-1-8,4504,8,0.0,0.0,3,BCL7B,0,4504,1786.0,0,...,0.248959,0.108813,-0.109756,-0.078059,-0.038733,-0.153273,-0.102016,-0.092018,-0.135454,-0.137559
AGAAGCGTCAGGACGA-1-9,4964,9,0.0,0.0,4,PBRM1,0,4963,2287.0,3,...,-0.124360,0.092573,-0.120545,-0.052604,-0.071641,-0.061278,-0.111678,-0.107756,-0.133673,-0.133523
GTTGCGGCAACCCGCA-1-6,6043,6,0.0,0.0,1,BCL7A,0,6043,3493.0,3,...,0.045556,0.042191,-0.087506,-0.045139,-0.055465,0.004878,-0.104217,-0.095756,-0.132171,-0.180121
TCTTTGAGTCAGTTTG-1-6,4472,6,0.0,0.0,3,SS18,0,4472,1646.0,4,...,0.304349,0.027444,-0.038663,-0.064119,-0.075211,-0.041830,-0.099594,-0.116923,-0.120978,-0.112966


In [23]:
layer_names = adata.layers.keys()
print("Layers in AnnData object:", list(layer_names))
# Inspect each layer's data matrix
for layer_name in layer_names:
    print(f"Layer '{layer_name}' data matrix:")
    print(adata.layers[layer_name])

Layers in AnnData object: ['counts', 'X_pert']
Layer 'counts' data matrix:
  (0, 3)	1.0
  (0, 4)	1.0
  (0, 8)	2.0
  (0, 12)	7.0
  (0, 15)	1.0
  (0, 16)	4.0
  (0, 18)	1.0
  (0, 23)	1.0
  (0, 25)	1.0
  (0, 26)	5.0
  (0, 28)	2.0
  (0, 35)	1.0
  (0, 42)	2.0
  (0, 50)	1.0
  (0, 53)	1.0
  (0, 58)	4.0
  (0, 61)	2.0
  (0, 62)	1.0
  (0, 66)	1.0
  (0, 68)	1.0
  (0, 70)	1.0
  (0, 73)	1.0
  (0, 82)	1.0
  (0, 85)	11.0
  (0, 86)	6.0
  :	:
  (38327, 1229)	7.0
  (38327, 1230)	4.0
  (38327, 1234)	1.0
  (38327, 1235)	3.0
  (38327, 1236)	1.0
  (38327, 1239)	23.0
  (38327, 1240)	1.0
  (38327, 1245)	4.0
  (38327, 1248)	2.0
  (38327, 1252)	1.0
  (38327, 1253)	1.0
  (38327, 1254)	2.0
  (38327, 1255)	11.0
  (38327, 1259)	1.0
  (38327, 1264)	14.0
  (38327, 1266)	4.0
  (38327, 1268)	1.0
  (38327, 1270)	2.0
  (38327, 1271)	2.0
  (38327, 1272)	22.0
  (38327, 1273)	4.0
  (38327, 1274)	1.0
  (38327, 1275)	2.0
  (38327, 1276)	1.0
  (38327, 1283)	1.0
Layer 'X_pert' data matrix:
[[        nan         nan         nan .

In [16]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, subset=True)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scanpy/preprocessing/_simple.py:377: RuntimeWarning: invalid value encountered in log1p
  np.log1p(X, out=X)


ValueError: Bin edges must be unique: Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan],
      dtype='float64').
You can drop duplicate edges by setting the 'duplicates' kwarg

In [17]:
ms = pt.tl.Mixscape()

In [18]:
ms.perturbation_signature(adata, "guide.compact", "unassigned", "batch")


In [19]:
adata_pert = adata.copy()

In [20]:
adata_pert.X = adata_pert.layers["X_pert"]

In [21]:
sc.pp.pca(adata_pert)
sc.pp.neighbors(adata_pert, metric="cosine")
sc.tl.umap(adata_pert)
sc.pl.umap(adata_pert, color=["batch", "guide.compact"])

ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [26]:
adata.layers["X_pert"] = adata.X.copy()
sc.pp.normalize_total(adata, layer="X_pert")
sc.pp.log1p(adata, layer="X_pert")
ms.mixscape(adata=adata, control="unassigned", labels="gene.compact", layer="X_pert")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scanpy/preprocessing/_normalization.py:233: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scanpy/preprocessing/_simple.py:377: RuntimeWarning: invalid value encountered in log1p
  np.log1p(X, out=X)


ValueError: Cannot specify `layer` and have `use_raw=True`.

In [1]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "dc90f93f-f6bd-489a-a688-9489246ceef3",
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "import tempfile\n",
    "\n",
    "import anndata as ad\n",
    "import muon\n",
    "import numpy as np\n",
    "import scanpy as sc\n",
    "import seaborn as sns\n",
    "import torch\n",
    "import muon as mu\n",
    "import pertpy as pt\n",
    "import pandas as pd\n",
    "import re\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "be2c1aa6",
   "metadata": {},
   "outputs": [],
   "source": [
    "import scvi\n",
    "print(scvi.__version__)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "0fc7f15b-03c2-43a2-afb7-073e61f4c5a5",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load the .h5ad file\n",
    "adata = ad.read_h5ad('/Users/ismail/thesis/Baf.single_gene.guide_qc.lm.h5ad')\n",
    "\n",
    "# Display the object\n",
    "print(adata)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "5a32b96d-c0f2-4b9a-b193-4f62df66391e",
   "metadata": {},
   "outputs": [],
   "source": [
    "adata.obs"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "d45f4580-5579-474c-be9a-733e00ff0e3a",
   "metadata": {},
   "outputs": [],
   "source": [
    "adata.var"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "aff52a93",
   "metadata": {},
   "outputs": [],
   "source": [
    "adata.X = adata.layers['counts']"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "ff4e150d-f0ae-4a1b-ad47-0a10cff70b6e",
   "metadata": {},
   "outputs": [],
   "source": [
    "def extract_guide_number(guide):\n",
    "    if guide == 'unassigned':\n",
    "        return guide\n",
    "    else:\n",
    "        # Extract the numeric part after the underscore\n",
    "        match = re.search(r'_(\\d+)', guide)\n",
    "        if match:\n",
    "            return match.group(1)\n",
    "        else:\n",
    "            return np.nan\n",
    "\n",
    "adata.obs['replicate'] = adata.obs['guide.compact'].apply(extract_guide_number)\n",
    "adata.obs[\"replicate\"]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "4cb17c69-3b72-442f-8d3c-751ee95f3f91",
   "metadata": {},
   "outputs": [],
   "source": [
    "adata.obs[\"gene.compact\"]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "b20e156d-d2ca-4945-80e8-0b7474b3c766",
   "metadata": {},
   "outputs": [],
   "source": [
    "perturbed_adata = adata[\n",
    "    adata.obs[\"gene.compact\"] != \"unassigned\"\n",
    "]  # Only consider perturbed cells\n",
    "perturbed_adata.obs\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "27d374b1-d8c2-4f43-acc4-dfe44400b62d",
   "metadata": {},
   "outputs": [],
   "source": [
    "# layer_names = adata.layers.keys()\n",
    "# print(\"Layers in AnnData object:\", list(layer_names))\n",
    "# # Inspect each layer's data matrix\n",
    "# for layer_name in layer_names:\n",
    "#     print(f\"Layer '{layer_name}' data matrix:\")\n",
    "#     print(adata.layers[layer_name])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "8ebe3ca3-3a7d-4bc4-b040-0cff7b99f780",
   "metadata": {},
   "outputs": [],
   "source": [
    "sc.pp.normalize_total(adata)\n",
    "sc.pp.log1p(adata)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "ee59325f-b7a0-454f-9f21-0e4d860c0de4",
   "metadata": {},
   "outputs": [],
   "source": [
    "ms = pt.tl.Mixscape()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "cab1de18-3052-46b9-850e-7691b764f3c1",
   "metadata": {},
   "outputs": [],
   "source": [
    "ms.perturbation_signature(adata, \"guide.compact\", \"unassigned\", \"replicate\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "0379c952-6c90-4c74-ae3b-59ccd7085e4f",
   "metadata": {},
   "outputs": [],
   "source": [
    "adata_pert = adata.copy()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "5605f4c5-aef0-48e3-8899-3535a0b7a506",
   "metadata": {},
   "outputs": [],
   "source": [
    "adata_pert.X = adata_pert.layers[\"X_pert\"]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "ea0ef0ac-61cf-4400-aa6b-9a2b551d0685",
   "metadata": {},
   "outputs": [],
   "source": [
    "sc.pp.pca(adata_pert)\n",
    "sc.pp.neighbors(adata_pert, metric=\"cosine\")\n",
    "sc.tl.umap(adata_pert)\n",
    "sc.pl.umap(adata_pert, color=[\"batch\", \"guide.compact\"])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "d104f652-887e-4e0f-80fe-3a16393176e7",
   "metadata": {},
   "outputs": [],
   "source": [
    "ms.mixscape(adata=adata, control=\"unassigned\", labels=\"gene.compact\", layer=\"X_pert\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "57ade88d-0b42-4260-bb6b-bd45bae497fc",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}

NameError: name 'null' is not defined